<h1><strong><u>K-Nearest Neighbors Model Selection</u></strong></h1>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, GridSearchCV, validation_curve
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB, CategoricalNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

<h2><u>Data Loading</u></h2>

In [ ]:
df = pd.DataFrame(columns=["title", "target"])

In [ ]:
# Formatting text files
titles = []
targets = []
file_path_dict = {'clickbait': './clickbait_data.txt', 'non clickbait': './non_clickbait_data.txt'}
for key, value in file_path_dict.items():
    with open(value, 'r') as file:
        for line_number, line in enumerate(file):
            line = line.strip()
            if line != "":
                titles.append(line)
                targets.append(key)
data_dict = {"title": titles, "target": targets}
df = pd.DataFrame(data_dict)


In [ ]:
data = df.sample(frac=1).reset_index(drop=True)
display(data)

,title,target
0,What Percent KatyCat Are You,clickbait
1,Rwandan army officer sentenced to 25 years for...,non clickbait
2,17 Times Kourtney Kardashian Shut Down Her Own...,clickbait
3,Canada's Conservatives launch new ad campaign,non clickbait
4,17 Makeup Facts That Show Humans Will Do Anyth...,clickbait
...,...,...
31995,"Andrew Garfield Has Transformed From One ""Harr...",clickbait
31996,Work on Lehman Brothers’ rescue to continue ov...,non clickbait
31997,13 Beautifully Honest Valentine's Cards For Pe...,clickbait
31998,17 Awkward Moments When You're A Jew On Christmas,clickbait


In [ ]:
X = data["title"]
y = data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
display(X_train.shape) 
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(25600,)

(6400,)

(25600,)

(6400,)

In [ ]:
def custom_tokenizer(text):
        from nltk.tokenize import word_tokenize
        from nltk.corpus import stopwords
        from nltk.stem import WordNetLemmatizer
        import re
        en_stopwords = stopwords.words('english')
        lemmatizer = WordNetLemmatizer()
        new_text = text.lower() #lowercase

        new_text = re.sub(r"([^\w\s])", "", new_text) #remove punctuation

        for word in new_text.split(): #remove stopwords
            if word in en_stopwords:
                new_text = new_text.replace(word, "")
        
        new_text = word_tokenize(new_text) #tokenize

        new_text = [lemmatizer.lemmatize(token) for token in new_text] #lemmatize
        return new_text

<h2><strong><u>Naive Bayes Model Selection</u></strong></h2>

I believe that the <strong>BernoulliNB</strong> model is the best Naive Bayes model for this dataset because it is designed for binary/boolean features.

<h3><u>Validation Curves</u></h3>